<a href="https://colab.research.google.com/github/T-adnane/NLP-Q-A/blob/main/NLP_Question_Answering_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

## **Prétraitement du text**

In [ ]:
# fonction pour faire la tokenisation
def tok_func(data):
  data = nltk.word_tokenize(data)
  return data

In [ ]:
# fonction pour la suppression des stopWords
def stpword_func(data):
  stopwords_en = set(stopwords.words('english'))
  data = [w for w in data if w not in stopwords_en]
  return data

In [ ]:
# fonction pour transformer le text en minuscule
def min_fonc(data):
  data = [w.lower() for w in data]
  return data

In [ ]:
# fonction pour faire steaming
def stem_func(data):
  stem = PorterStemmer()
  data = [stem.stem(w) for w in data]
  return data

In [ ]:
# fonction pour clean_text
def clean_func(data):
  data = tok_func(data)
  data = stpword_func(data)
  data = min_fonc(data)
  data = stem_func(data)
  data = " ".join(data)
  return data

In [ ]:
import nltk
from nltk.corpus import stopwords
import string
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import textwrap
from nltk.stem import PorterStemmer
from sklearn.preprocessing import OneHotEncoder
import numpy as np
nltk.download('punkt')
nltk.download('stopwords')

# **Cosinus Simularity**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def answer_question(context, question):
    context = clean_func(context)
    question = clean_func(question)
    # Sépare le contexte en phrases
    sentences = context.split('. ')

    # Initialise le vectoriseur
    vectorizer = CountVectorizer(stop_words='english')

    # Transforme le contexte en matrice de fréquences de mots
    X = vectorizer.fit_transform(sentences)

    # Transforme la question en vecteur de fréquences de mots (Bag of words)
    question_vec = vectorizer.transform([question])

    # Calcule les similarités cosinus entre la question et les phrases du contexte
    similarities = cosine_similarity(X, question_vec)

    # Trouve l'indice de la phrase la plus similaire à la question
    best_match_index = similarities.argmax()

    # Récupère la phrase la plus similaire
    best_match = sentences[best_match_index]

    # Retourne la phrase comme réponse à la question
    return best_match


In [ ]:
context = "je suis adnane. le capital du maroc c'est rabat. il fait chaud"
question = "qu'elle est le capital du maroc ?"

answer = answer_question(context, question)
print(answer)


le capit du maroc c'est rabat 


# **Bert**

## **Install and Import Lib**

## **Q&A Model**

In [ ]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:
encoder = OneHotEncoder()

In [ ]:
def answer_question(question, answer_text):
    # Encode la question et le texte de réponse en utilisant le tokenizer
    input_ids = tokenizer.encode(question, answer_text)

    # Trouve l'index du token de séparation [SEP]
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # Calcule le nombre de tokens dans la première et deuxième séquence
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a

    # Crée les ids de segment pour chaque token
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # Vérifie que la longueur des ids de segment est la même que celle des input_ids
    assert len(segment_ids) == len(input_ids)

    # Effectue la prédiction en utilisant le modèle
    outputs = model(torch.tensor([input_ids]), 
                    token_type_ids=torch.tensor([segment_ids]), 
                    return_dict=True)

    # Récupère les scores de début et de fin de réponse
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # Trouve l'index du token de début et de fin de réponse avec les scores max
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Convertit les ids de tokens en tokens réels
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Concatène les tokens pour former la réponse
    answer = tokens[answer_start]
    for i in range(answer_start + 1, answer_end + 1):
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        else:
            answer += ' ' + tokens[i]

    # Affiche la réponse
    print('Réponse : "' + answer + '"')


## **tester le modéle**

### **answer_text**

In [ ]:
wrapper = textwrap.TextWrapper(width=80) 
bert_abstract = "Musk was born in Pretoria, South Africa, and briefly attended at the University of Pretoria before moving to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University and transferred to the University of Pennsylvania, where he received bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University. After two days, he dropped out and, with his brother Kimbal, co-founded the online city guide software company Zip2. In 1999, Zip2 was acquired by Compaq for $307 million and Musk co-founded X.com, a direct bank. X.com merged with Confinity in 2000 to form PayPal, which eBay acquired for $1.5 billion in 2002. Musk received an EB-5 investor green card in 1997, which led to his U.S. citizenship in 2002.[8]"
bert_abstract = clean_func(bert_abstract)
print(wrapper.fill(bert_abstract))

musk born pretoria , south africa , briefli attend univers pretoria move canada
age 18 , acquir citizenship canadian-born mother . two year later , matricul
queen 's univers transfer univers pennsylvania , receiv bachelor 's degre econom
physic . he move california 1995 attend stanford univers . after two day , drop
, brother kimbal , co-found onlin citi guid softwar compani zip2 . in 1999 ,
zip2 acquir compaq $ 307 million musk co-found x.com , direct bank . x.com merg
confin 2000 form paypal , ebay acquir $ 1.5 billion 2002 . musk receiv eb-5
investor green card 1997 , led u.s. citizenship 2002 . [ 8 ]


### **question**

In [ ]:
question = "where musk is born?"
question = clean_func(question)
answer_question(question, bert_abstract)

Réponse : "pretoria , south africa"
